In [1]:
korean_cities = {
    "Seoul": (37.5665, 126.9780),
    "Busan": (35.1796, 129.0756),
    "Incheon": (37.4563, 126.7052),
    "Daegu": (35.8714, 128.6014),
    "Daejeon": (36.3504, 127.3845),
    "Gwangju": (35.1595, 126.8526),
    "Suwon": (37.2636, 127.0286),
    "Ulsan": (35.5384, 129.3114),
    "Changwon": (35.2285, 128.6811),
    "Jeonju": (35.8242, 127.1480),
}

In [2]:
import math

def haversine(coord1, coord2):
    R = 6371  # Earth radius (km)
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c  # km

COST_PER_KM = 100

def travel_cost(city1, city2):
    dist = haversine(korean_cities[city1], korean_cities[city2])
    return dist * COST_PER_KM

Breadth-First Search

In [3]:
import time
from collections import deque

def compute_all_distances(cities):
    city_list = list(cities.keys())
    n = len(city_list)
    dist = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                c1, c2 = city_list[i], city_list[j]
                dist[(c1, c2)] = travel_cost(c1, c2)
    return dist

def bfs_tsp(start_city, cities):
    start_time = time.time()
    city_list = list(cities.keys())
    n = len(city_list)
    dist = compute_all_distances(cities)

    queue = deque()
    queue.append((start_city, [start_city], 0)) 
    min_cost = float('inf')
    best_path = None

    while queue:
        current_city, path, cost = queue.popleft()

        if len(path) == n:
            # Return to start to complete TSP
            total_cost = cost + dist[(current_city, start_city)]
            if total_cost < min_cost:
                min_cost = total_cost
                best_path = path + [start_city]
            continue

        for next_city in city_list:
            if next_city not in path:
                new_cost = cost + dist[(current_city, next_city)]
                queue.append((next_city, path + [next_city], new_cost))

    duration = time.time() - start_time
    return best_path, min_cost, duration

In [4]:
path, cost, elapsed = bfs_tsp("Seoul", korean_cities)
print("BFS TSP Result:")
print("Path:", " → ".join(path))
print("Total Cost (KRW):", round(cost, 2))
print("Execution Time (s):", round(elapsed, 4))

BFS TSP Result:
Path: Seoul → Daegu → Ulsan → Busan → Changwon → Gwangju → Jeonju → Daejeon → Suwon → Incheon → Seoul
Total Cost (KRW): 86944.15
Execution Time (s): 0.8248


Uniform-Cost Search

In [5]:
import heapq

def ucs_tsp(start_city, cities):
    start_time = time.time()
    city_list = list(cities.keys())
    n = len(city_list)
    dist = compute_all_distances(cities)
    
    frontier = [(0, start_city, [start_city])]
    min_cost = float('inf')
    best_path = None

    while frontier:
        cost, current_city, path = heapq.heappop(frontier)

        if len(path) == n:
            total_cost = cost + dist[(current_city, start_city)]
            if total_cost < min_cost:
                min_cost = total_cost
                best_path = path + [start_city]
            continue

        for next_city in city_list:
            if next_city not in path:
                new_cost = cost + dist[(current_city, next_city)]
                heapq.heappush(frontier, (new_cost, next_city, path + [next_city]))

    duration = time.time() - start_time
    return best_path, min_cost, duration

In [6]:
path, cost, elapsed = ucs_tsp("Seoul", korean_cities)
print("UCS TSP Result:")
print("Path:", " → ".join(path))
print("Total Cost (KRW):", round(cost, 2))
print("Execution Time (s):", round(elapsed, 4))

UCS TSP Result:
Path: Seoul → Daegu → Ulsan → Busan → Changwon → Gwangju → Jeonju → Daejeon → Suwon → Incheon → Seoul
Total Cost (KRW): 86944.15
Execution Time (s): 2.0457


Depth-First Search

In [7]:
def dfs_tsp(start_city, cities):
    start_time = time.time()
    city_list = list(cities.keys())
    n = len(city_list)
    dist = compute_all_distances(cities)

    stack = [(start_city, [start_city], 0)]
    min_cost = float('inf')
    best_path = None

    while stack:
        current_city, path, cost = stack.pop()

        if len(path) == n:
            total_cost = cost + dist[(current_city, start_city)]
            if total_cost < min_cost:
                min_cost = total_cost
                best_path = path + [start_city]
            continue

        for next_city in reversed(city_list):  # DFS → LIFO 순서
            if next_city not in path:
                new_cost = cost + dist[(current_city, next_city)]
                stack.append((next_city, path + [next_city], new_cost))

    duration = time.time() - start_time
    return best_path, min_cost, duration

In [8]:
path, cost, elapsed = dfs_tsp("Seoul", korean_cities)
print("DFS TSP Result:")
print("Path:", " → ".join(path))
print("Total Cost (KRW):", round(cost, 2))
print("Execution Time (s):", round(elapsed, 4))

DFS TSP Result:
Path: Seoul → Daegu → Ulsan → Busan → Changwon → Gwangju → Jeonju → Daejeon → Suwon → Incheon → Seoul
Total Cost (KRW): 86944.15
Execution Time (s): 0.5292


Depth-Limited Search

In [9]:
subset_cities = {
    "Seoul": korean_cities["Seoul"],
    "Incheon": korean_cities["Incheon"],
    "Suwon": korean_cities["Suwon"],
    "Daejeon": korean_cities["Daejeon"],
    "Jeonju": korean_cities["Jeonju"],
    "Daegu": korean_cities["Daegu"]
}

In [10]:
def dls_tsp(start_city, cities, depth_limit):
    start_time = time.time()
    city_list = list(cities.keys())
    dist = compute_all_distances(cities)
    min_cost = float('inf')
    best_path = None

    def dfs(city, path, cost, depth):
        nonlocal min_cost, best_path
        if depth == depth_limit:
            total_cost = cost + dist[(city, start_city)]
            if total_cost < min_cost:
                min_cost = total_cost
                best_path = path + [start_city]
            return

        for next_city in city_list:
            if next_city not in path:
                dfs(next_city, path + [next_city], cost + dist[(city, next_city)], depth + 1)

    dfs(start_city, [start_city], 0, 0)
    duration = time.time() - start_time
    return best_path, min_cost, duration

In [11]:
path, cost, elapsed = dls_tsp("Seoul", subset_cities, depth_limit=5)
print("DLS TSP Result (6 cities):")
print("Path:", " → ".join(path))
print("Total Cost (KRW):", round(cost, 2))
print("Execution Time (s):", round(elapsed, 4))

DLS TSP Result (6 cities):
Path: Seoul → Incheon → Suwon → Daejeon → Jeonju → Daegu → Seoul
Total Cost (KRW): 60006.08
Execution Time (s): 0.0001


Iterative Deepening Search

In [12]:
def ids_tsp(start_city, cities):
    start_time = time.time()
    city_list = list(cities.keys())
    n = len(city_list)
    dist = compute_all_distances(cities)

    min_cost = float('inf')
    best_path = None

    def dls(city, path, cost, depth_limit):
        nonlocal min_cost, best_path
        if len(path) > depth_limit:
            return
        if len(path) == n:
            total_cost = cost + dist[(city, start_city)]
            if total_cost < min_cost:
                min_cost = total_cost
                best_path = path + [start_city]
            return
        for next_city in city_list:
            if next_city not in path:
                dls(next_city, path + [next_city], cost + dist[(city, next_city)], depth_limit)

    for limit in range(1, n + 1):
        dls(start_city, [start_city], 0, limit)

    duration = time.time() - start_time
    return best_path, min_cost, duration

In [13]:
path, cost, elapsed = ids_tsp("Seoul", korean_cities)
print("IDS TSP Result:")
print("Path:", " → ".join(path))
print("Total Cost (KRW):", round(cost, 2))
print("Execution Time (s):", round(elapsed, 4))

IDS TSP Result:
Path: Seoul → Daegu → Ulsan → Busan → Changwon → Gwangju → Jeonju → Daejeon → Suwon → Incheon → Seoul
Total Cost (KRW): 86944.15
Execution Time (s): 1.4496


Bidirectional Search

In [17]:
from itertools import permutations
import time

def bidirectional_tsp(start_city, cities):
    start_time = time.time()
    city_list = list(cities.keys())
    city_list.remove(start_city)
    n = len(city_list)

    dist = compute_all_distances(cities)

    min_cost = float('inf')
    best_path = None

    for perm in permutations(city_list):
        path = [start_city] + list(perm) + [start_city]
        cost = 0
        for i in range(len(path) - 1):
            cost += dist[(path[i], path[i + 1])]
        if cost < min_cost:
            min_cost = cost
            best_path = path

    duration = time.time() - start_time
    return best_path, min_cost, duration

In [18]:
path, cost, elapsed = bidirectional_tsp("Seoul", korean_cities)
print("Bidirectional TSP Result:")
print("Path:", " → ".join(path))
print("Total Cost (KRW):", round(cost, 2))
print("Execution Time (s):", round(elapsed, 4))

Bidirectional TSP Result:
Path: Seoul → Daegu → Ulsan → Busan → Changwon → Gwangju → Jeonju → Daejeon → Suwon → Incheon → Seoul
Total Cost (KRW): 86944.15
Execution Time (s): 0.4424


Comparative Study

In [19]:
import pandas as pd

results_summary = [
    {"Algorithm": "Breadth-First Search", "Path": bfs_tsp("Seoul", korean_cities)[0],
     "Cost": bfs_tsp("Seoul", korean_cities)[1], "Time (s)": round(bfs_tsp("Seoul", korean_cities)[2], 4)},
    
    {"Algorithm": "Uniform-Cost Search", "Path": ucs_tsp("Seoul", korean_cities)[0],
     "Cost": ucs_tsp("Seoul", korean_cities)[1], "Time (s)": round(ucs_tsp("Seoul", korean_cities)[2], 4)},
    
    {"Algorithm": "Depth-First Search", "Path": dfs_tsp("Seoul", korean_cities)[0],
     "Cost": dfs_tsp("Seoul", korean_cities)[1], "Time (s)": round(dfs_tsp("Seoul", korean_cities)[2], 4)},
    
    {"Algorithm": "Depth-Limited Search", "Path": dls_tsp("Seoul", subset_cities, 5)[0],
     "Cost": dls_tsp("Seoul", subset_cities, 5)[1], "Time (s)": round(dls_tsp("Seoul", subset_cities, 5)[2], 4)},
    
    {"Algorithm": "Iterative Deepening Search", "Path": ids_tsp("Seoul", korean_cities)[0],
     "Cost": ids_tsp("Seoul", korean_cities)[1], "Time (s)": round(ids_tsp("Seoul", korean_cities)[2], 4)},
    
    {"Algorithm": "Bidirectional Search", "Path": bidirectional_tsp("Seoul", korean_cities)[0],
     "Cost": bidirectional_tsp("Seoul", korean_cities)[1], "Time (s)": round(bidirectional_tsp("Seoul", korean_cities)[2], 4)},
]


df_results = pd.DataFrame(results_summary)
df_results["Path"] = df_results["Path"].apply(lambda p: " → ".join(p) if p else "None")
pd.DataFrame(df_results)

,Algorithm,Path,Cost,Time (s)
0,Breadth-First Search,Seoul → Daegu → Ulsan → Busan → Changwon → Gwa...,86944.154475,0.8980
1,Uniform-Cost Search,Seoul → Daegu → Ulsan → Busan → Changwon → Gwa...,86944.154475,2.2914
2,Depth-First Search,Seoul → Daegu → Ulsan → Busan → Changwon → Gwa...,86944.154475,0.5970
3,Depth-Limited Search,Seoul → Incheon → Suwon → Daejeon → Jeonju → D...,60006.075169,0.0001
4,Iterative Deepening Search,Seoul → Daegu → Ulsan → Busan → Changwon → Gwa...,86944.154475,1.3383
5,Bidirectional Search,Seoul → Daegu → Ulsan → Busan → Changwon → Gwa...,86944.154475,0.4060
